In [ ]:
!gdown --id 1L2_InO7XjjZtlBDDfzz2v3uCPodwucmJ
!unzip DeepSDF.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls

In [ ]:
!pip install trimesh

In [ ]:
!pip install pysdf

In [ ]:
!pip install plyfile

## Implicit Function (Signed Distance Function)
   

#### What is an Implicit Function?

An implicit function defines a shape by a condition on $(x, y)$, such as $f(x, y) = 0$. For a circle of radius $r$ centered at $(x_0, y_0)$, the implicit equation is:

$$x^2 + y^2 - r^2 = 0$$

The Signed Distance Function (SDF) gives the shortest distance from any point $(x, y)$ to the circle's boundary, with the sign indicating inside (negative), on (zero), or outside (positive) the circle.

In [ ]:
!pip install ipympl
get_ipython().kernel.do_shutdown(restart=True)



In [ ]:
%cd /content/DeepSDF

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
# from matplotlib.widgets import Slider
# %matplotlib widget


## Defining the Circle's Signed Distance Function (SDF)
The SDF for a circle centered at $(0, 0)$ with radius $r$ is:

$$f(x, y) = \sqrt{x^2 + y^2} - r$$

In [ ]:
def circle_sdf(x, y, r, a=0.2, k=6):
    return np.sqrt(x**2 + y**2) - r

## Setting up the Grid

We'll create a grid of $(x, y)$ points to evaluate and visualize the SDF.

In [ ]:
grid_size = 400
x = np.linspace(-2, 2, grid_size)
y = np.linspace(-2, 2, grid_size)
X, Y = np.meshgrid(x, y)

## Visualizing the SDF with an Interactive Slider

The plot below shows the SDF as a color map. The black contour represents the circle (where SDF = 0). Use the slider to change the radius and see how the SDF and the circle change.
    
- **Blue/Red colors**: Indicate the signed distance.
- **Black contour**: The actual circle boundary.
- **Colorbar**: Shows what the colors mean (signed distance values).

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
# Set up the interactive plotting function
def update(radius=1.0):
    Z = circle_sdf(X, Y, radius)

    fig, ax = plt.subplots(figsize=(6, 5))

    cmap = ax.imshow(Z, extent=[-2, 2, -2, 2], origin='lower', cmap='coolwarm', alpha=0.8)
    cbar = fig.colorbar(cmap, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Signed Distance')

    contour = ax.contour(X, Y, Z, levels=[0], colors='black')

    ax.set_title(f'Circle SDF Visualization (Radius={radius:.2f})')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    plt.show()

# Create an ipywidget slider for radius
radius_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=1.9,
    step=0.01,
    description='Radius:',
    continuous_update=False
)

# Bind the slider to the update function
ui = widgets.HBox([radius_slider])
out = widgets.interactive_output(update, {'radius': radius_slider})

display(ui, out)

In [ ]:
def ameoba_sdf(x, y, r, a=0.2, k=6):
    theta = np.arctan2(y, x)
    r = r + a * np.sin(k * theta)
    return np.sqrt(x**2 + y**2) - r


In [ ]:
# Set up the interactive plotting function
def update(radius=1.0):
    Z = ameoba_sdf(X, Y, radius)

    fig, ax = plt.subplots(figsize=(6, 5))

    cmap = ax.imshow(Z, extent=[-2, 2, -2, 2], origin='lower', cmap='coolwarm', alpha=0.8)
    cbar = fig.colorbar(cmap, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('Signed Distance')

    contour = ax.contour(X, Y, Z, levels=[0], colors='black')

    ax.set_title(f'Circle SDF Visualization (Radius={radius:.2f})')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

    plt.show()

# Create an ipywidget slider for radius
radius_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=1.9,
    step=0.01,
    description='Radius:',
    continuous_update=False
)

# Bind the slider to the update function
ui = widgets.HBox([radius_slider])
out = widgets.interactive_output(update, {'radius': radius_slider})

display(ui, out)

#### Meshes

In [ ]:
import trimesh
mesh_path = 'data/armadillo.obj'
scene = trimesh.scene.Scene()
scene.add_geometry(trimesh.load(mesh_path))
scene.show()

## Siren

In [ ]:

import sys
import os

from torch.utils.data import DataLoader, Dataset
import torch
import numpy as np
import trimesh
import pysdf
import torch.nn as nn
from tqdm.autonotebook import tqdm


In [ ]:
"""Extract mesh from an already optimised latent code and network.
Store the mesh in the same folder where the latent code is located."""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:


class MeshSDF(Dataset):
    def __init__(self, mesh_path='data/armadillo.obj', size=100, num_samples=2**16 ):
        super().__init__()

        print("Loading point cloud")

        self.mesh = trimesh.load(mesh_path)
        print("Finished loading point cloud")
        self.normals = self.mesh.vertex_normals

        self.sdf_fn = pysdf.SDF(self.mesh.vertices, self.mesh.faces)

        self.num_samples = num_samples
        assert self.num_samples % 8 == 0, "num_samples must be divisible by 8."

        self.size = size

    def __len__(self):
        return self.size


    def __getitem__(self, idx):

        sdfs = np.zeros((self.num_samples, 1))

        # surface
        points_surface = self.mesh.sample(self.num_samples * 7 // 8)
        # perturb surface
        points_surface[self.num_samples // 2:] += 0.05 * np.random.randn(self.num_samples * 3 // 8, 3)
        # random
        points_uniform = np.random.rand(self.num_samples // 8, 3) * 2 - 1
        points = np.concatenate([points_surface, points_uniform], axis=0).astype(np.float32)

        sdfs[self.num_samples // 2:] = -self.sdf_fn(points[self.num_samples // 2:])[:,None].astype(np.float32)


        coords = torch.from_numpy(points).float()
        return coords, torch.from_numpy(sdfs).float(), torch.from_numpy(self.normals).float()




In [ ]:
#define dataloader
sdf_dataset = MeshSDF()
dataloader = DataLoader(sdf_dataset, shuffle=True, batch_size=1, pin_memory=True, num_workers=0)


In [ ]:
class SineLayer(nn.Module):
    def __init__(self, in_features, out_features, bias=True, is_first=False, omega_0=30.):
        super().__init__()
        self.omega_0 = omega_0
        self.is_first = is_first
        self.in_features = in_features
        self.linear = nn.Linear(in_features, out_features, bias=bias)
        self.init_weights()


    def init_weights(self):
        with torch.no_grad():
            if self.is_first:
                self.linear.weight.uniform_(-1 / self.in_features, 1 / self.in_features)
            else:
                self.linear.weight.uniform_(-np.sqrt(6 / self.in_features) / self.omega_0, np.sqrt(6 / self.in_features) / self.omega_0)

    def forward(self, input):
        return torch.sin(self.omega_0 * self.linear(input))

class Siren(nn.Module):
    def __init__(self,in_features=3, hidden_features=256, hidden_layers=3, out_features=3, outermost_linear=True, first_omega_0=30., hidden_omega_0=30.):
        super().__init__()

        self.net = []
        self.net.append(SineLayer(in_features, hidden_features, is_first=True, omega_0=first_omega_0))

        for i in range(hidden_layers):
            self.net.append(SineLayer(hidden_features, hidden_features, is_first=False, omega_0=hidden_omega_0))

        if outermost_linear:
            final_linear = nn.Linear(hidden_features, out_features)
            with torch.no_grad():
                final_linear.weight.uniform_(-np.sqrt(6 / hidden_features) / hidden_omega_0, np.sqrt(6 / hidden_features) / hidden_omega_0)
            self.net.append(final_linear)
        else:
            self.net.append(SineLayer(hidden_features, out_features, is_first=False, omega_0=hidden_omega_0))
        self.net = nn.Sequential(*self.net)

    def forward(self, coords):
        output = self.net(coords)
        return output


In [ ]:
model = Siren(in_features=3, out_features=1)
model.to(device)
optimizer = torch.optim.Adam(lr=1e-4, params=model.parameters())


In [ ]:

def train(model, optimizer, train_dataloader):

    root_path = os.path.join('./siren_sdf')
    summaries_dir = os.path.join(root_path, 'summaries')
    checkpoints_dir = os.path.join(root_path, 'checkpoints')
    os.makedirs(root_path, exist_ok=True)
    os.makedirs(summaries_dir, exist_ok=True)
    os.makedirs(checkpoints_dir, exist_ok=True)


    epochs = 10000
    epochs_til_checkpoint = 1000

    total_steps = 0
    losses = { loss:[] for loss in ['loss']}


    with tqdm(total=len(train_dataloader) * epochs) as pbar:
        train_losses = []

        for epoch in range(epochs):
            if not epoch % epochs_til_checkpoint and epoch:
                torch.save(model.state_dict(),os.path.join(checkpoints_dir, 'model_epoch_%04d.pth' % epoch))

            for step, data in enumerate(train_dataloader):

                coords, gt_sdf, gt_normals = data[0],data[1],data[2]
                coords, gt_sdf, gt_normals = coords.to(device), gt_sdf.to(device), gt_normals.to(device)


                pred = model(coords)


                difference = (pred - gt_sdf).abs()
                scale = 1 / (gt_sdf.abs() + 1e-2)
                loss = difference * scale
                train_loss = loss.mean()

                log_train_loss = train_loss.detach().item()


                losses['loss'].append(log_train_loss)
                train_losses.append(train_loss.item())

                optimizer.zero_grad()
                train_loss.backward()

                torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.)
                optimizer.step()

                if not total_steps % epochs_til_checkpoint:
                    tqdm.write("Epoch %d, Train loss %0.6f" % (epoch, log_train_loss))
                    torch.save(model.state_dict(),os.path.join(checkpoints_dir, 'model_current.pth'))

                pbar.update(1)

                total_steps += 1

        torch.save(model.state_dict(),os.path.join(checkpoints_dir, 'model_final.pth'))




In [ ]:
train(model, optimizer,dataloader)


In [ ]:

import sdf_meshing
model = Siren(in_features=3, out_features=1)
model.to(device)

model.load_state_dict(torch.load('checkpoints/siren.pth'))

mesh_path = 'siren_sdf/recon'
siren_mesh = sdf_meshing.create_mesh(model, filename=mesh_path, N=256)
scene = trimesh.scene.Scene()
scene.add_geometry(trimesh.load(mesh_path+'.ply'))
scene.show()


## DeepSDF

In [ ]:
import torch
import os
import model.model_sdf as sdf_model
from utils import utils_deepsdf
import trimesh
from results import runs_sdf
import results
import numpy as np
import config_files
import yaml

In [ ]:
"""Extract mesh from an already optimised latent code and network.
Store the mesh in the same folder where the latent code is located."""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [ ]:

def read_params(cfg):
    """Read the settings from the settings.yaml file. These are the settings used during training."""
    training_settings_path = os.path.join('results/runs_sdf',  cfg['folder_sdf'], 'settings.yaml')
    with open(training_settings_path, 'rb') as f:
        training_settings = yaml.load(f, Loader=yaml.FullLoader)

    return training_settings


In [ ]:

def reconstruct_object(cfg, latent_code, obj_idx, model, coords_batches, grad_size_axis):
    """
    Reconstruct the object from the latent code and save the mesh.
    Meshes are stored as .obj files under the same folder cerated during training, for example:
    - runs_sdf/<datetime>/meshes_training/mesh_0.obj
    """
    sdf = utils_deepsdf.predict_sdf(latent_code, coords_batches, model)
    try:
        vertices, faces = utils_deepsdf.extract_mesh(grad_size_axis, sdf)
    except:
        print('Mesh extraction failed')
        return

    # save mesh as obj
    mesh_dir = os.path.join('results/runs_sdf',  cfg['folder_sdf'], 'meshes_training')
    if not os.path.exists(mesh_dir):
        os.mkdir(mesh_dir)
    obj_path = os.path.join(mesh_dir, f"mesh_{obj_idx}.obj")
    trimesh.exchange.export.export_mesh(trimesh.Trimesh(vertices, faces), obj_path, file_type='obj')



In [ ]:
cfg_path = os.path.join(os.path.dirname(config_files.__file__), 'reconstruct_from_latent.yaml')
with open(cfg_path, 'rb') as f:
    cfg = yaml.load(f, Loader=yaml.FullLoader)


training_settings = read_params(cfg)

# Load the model
weights = os.path.join('results/runs_sdf',  cfg['folder_sdf'], 'weights.pt')

model = sdf_model.SDFModel(
    num_layers=training_settings['num_layers'],
    skip_connections=training_settings['latent_size'],
    latent_size=training_settings['latent_size'],
    inner_dim=training_settings['inner_dim']).to(device)
model.load_state_dict(torch.load(weights, map_location=device))

# Extract mesh obtained with the latent code optimised at inference
coords, grad_size_axis = utils_deepsdf.get_volume_coords(cfg['resolution'])
coords = coords.to(device)

# Split coords into batches because of memory limitations
coords_batches = torch.split(coords, 100000)

# Load paths
str2int_path = os.path.join(os.path.dirname(results.__file__), 'idx_str2int_dict.npy')
results_dict_path = os.path.join('results/runs_sdf',  cfg['folder_sdf'], 'results.npy')

# Load dictionaries
str2int_dict = np.load(str2int_path, allow_pickle=True).item()
results_dict = np.load(results_dict_path, allow_pickle=True).item()

for obj_id_path in cfg['obj_ids']:
    # Get object index in the results dictionary
    obj_idx = str2int_dict[obj_id_path]  # index in collected latent vector
    # Get the latent code optimised during training
    latent_code = results_dict['best_latent_codes'][obj_idx]
    latent_code = torch.tensor(latent_code).to(device)
    reconstruct_object(cfg, latent_code, obj_idx, model, coords_batches, grad_size_axis)


In [ ]:
scene = trimesh.scene.Scene()
scene.add_geometry(trimesh.load('results/runs_sdf/17_07_172540/meshes_training/mesh_1.obj'))
scene.show()